In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
#pm.help()
#pm.reset()
pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df_jo = pm.df('df_jo')
#df_jo.info()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
df_jo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32094 entries, 0 to 32093
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id_cr                       32094 non-null  int64          
 1   amount                      32094 non-null  float64        
 2   stat_cr                     32094 non-null  object         
 3   created_at                  32094 non-null  datetime64[ns] 
 4   user_id                     32094 non-null  int64          
 5   active                      32094 non-null  int64          
 6   moderated_at                21759 non-null  datetime64[ns] 
 7   reimbursement_date          32094 non-null  datetime64[ns] 
 8   cash_request_received_date  24149 non-null  datetime64[ns] 
 9   money_back_date             23917 non-null  datetime64[ns] 
 10  transfer_type               32094 non-null  object         
 11  send_at                     22678 non-nul

In [6]:
df = pm.df('df_jo')
#df.info()

cr_id = ['id_cr','id_fe','user_id', 'created_at','created_at_fe','amount','fee','stat_cr','stat_fe','transfer_type','type',
            'to_receive_ini', 'to_receive_bank','to_reimbur','to_reimbur_cash','to_end','to_send',
             'send_at', 'cr_received_date', 'money_back_date', 'reimbursement_date',
            'paid_at','charge_moment','moderated_at','reason','category','from_date','to_date', 'recovery_status'] # ,'id_y','from_date','to_date','reason','recovery_status', 'cash_request_id'

print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-18264]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[cr_id].sort_values('created_at'))

user_ids = [54879, 9900458, 2002, 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

pd.options.display.max_columns = None
print("Casos segun Cash User ID")
for id in (user_ids):
    #df_t = df[df['user_id'] == id].reset_index()
    df_t = df[(df['user_id'] == id) & (df['stat_cr'] == 'money_back')].reset_index()
    print(f"User ID: {id}")
    display(df_t[cr_id].sort_values('created_at').reset_index())

Casos segun Cash Request ID
Cash Request ID: -18264


,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status


Casos segun Cash User ID
User ID: 54879


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status


User ID: 9900458


...

,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,3,314,0,90,2019-12-26 19:20:47.806186,NaT,100.0,NaN,money_back,NaN,regular,NaN,4 days 04:39:12.193814,NaT,13 days 03:39:12.193814,NaT,NaT,NaT,NaT,2019-12-31,NaT,2020-01-08 23:00:00.000000,NaT,NaN,2019-12-27 14:06:27.008412,NaN,NaN,NaT,NaT,nice
1,4,667,0,90,2020-02-11 19:04:59.715551,NaT,100.0,NaN,money_back,NaN,regular,NaN,0 days 04:55:00.284449,NaT,19 days 03:55:00.284449,NaT,-9 days +01:29:24.798386,NaT,NaT,2020-02-12,2020-03-10 21:30:35.201614,2020-03-01 23:00:00.000000,NaT,NaN,2020-02-12 11:17:31.780167,NaN,NaN,NaT,NaT,nice
2,1,893,0,90,2020-03-16 14:56:43.473601,NaT,100.0,NaN,money_back,NaN,regular,NaN,1 days 09:03:16.526399,NaT,21 days 07:03:16.526399,NaT,-9 days +01:08:59.185284,NaT,NaT,2020-03-18,2020-04-15 20:51:00.814716,2020-04-06 22:00:00.000000,NaT,NaN,2020-03-16 15:11:42.665961,NaN,NaN,NaT,NaT,nice
3,0,2155,0,90,2020-05-25 12:48:13.019894,NaT,100.0,NaN,money_back,NaN,regular,NaN,2 days 11:11:46.980106,NaT,11 days 09:11:46.980106,NaT,-7 days +23:22:01.202196,NaT,NaT,2020-05-28,2020-06-11 22:37:58.797804,2020-06-05 22:00:00.000000,NaT,NaN,2020-05-25 13:20:08.786743,NaN,NaN,NaT,NaT,nice
4,2,3343,0,90,2020-06-15 10:39:34.100529,NaT,100.0,NaN,money_back,NaN,regular,NaN,2 days 13:20:25.899471,NaT,9 days 23:59:59.991291,NaT,-2 days +07:14:00.005528,NaT,NaT,2020-06-18,2020-06-27 03:25:34.086292,2020-06-25 10:39:34.091820,NaT,NaN,2020-06-15 11:52:28.283050,NaN,NaN,NaT,NaT,nice
5,5,5608,0,90,2020-06-27 14:08:39.137822,NaT,100.0,NaN,money_back,NaN,regular,NaN,2 days 09:51:20.862178,NaT,11 days 09:42:20.862178,NaT,-1 days +01:51:00,NaT,NaT,2020-06-30,2020-07-09 22:00:00.000000,2020-07-08 23:51:00.000000,NaT,NaN,2020-06-27 17:43:48.707336,NaN,NaN,NaT,NaT,nice
6,8,9206,0,90,2020-07-21 08:16:11.513030,NaT,100.0,NaN,money_back,NaN,regular,NaN,1 days 15:43:48.486970,-6 days +15:43:48.487391,9 days 13:43:48.486970,2 days 13:43:48.487391,0 days 00:00:00,6 days 23:59:59.999579,2020-07-28 08:16:11.512609,2020-07-23,2020-07-30 22:00:00.000000,2020-07-30 22:00:00.000000,NaT,NaN,2020-07-21 13:49:36.592679,NaN,NaN,NaT,NaT,nice
7,6,11384,0,90,2020-08-07 14:50:30.098500,NaT,100.0,NaN,money_back,NaN,regular,NaN,10 days 09:09:29.901500,3 days 09:09:29.902154,21 days 07:09:29.901500,14 days 07:09:29.902154,-2 days +00:00:00,6 days 23:59:59.999346,2020-08-14 14:50:30.097846,2020-08-18,2020-08-30 22:00:00.000000,2020-08-28 22:00:00.000000,NaT,NaN,2020-08-07 15:58:06.719912,NaN,NaN,NaT,NaT,nice
8,7,14367,0,90,2020-09-03 12:54:09.185674,NaT,100.0,NaN,money_back,NaN,regular,NaN,8 days 11:05:50.814326,1 days 11:05:50.883830,28 days 09:05:50.814326,21 days 09:05:50.883830,-1 days +10:33:46.582802,6 days 23:59:59.930496,2020-09-10 12:54:09.116170,2020-09-12,2020-10-02 11:26:13.417198,2020-10-01 22:00:00.000000,NaT,NaN,NaT,NaN,NaN,NaT,NaT,nice
9,9,19294,0,90,2020-10-05 15:34:36.390378,NaT,100.0,NaN,money_back,NaN,regular,NaN,8 days 08:25:23.609622,1 days 08:25:23.654154,28 days 06:25:23.609622,21 days 06:25:23.654154,0 days 02:22:19.535709,6 days 23:59:59.955468,2020-10-12 15:34:36.345846,2020-10-14,2020-11-02 19:37:40.464291,2020-11-02 22:00:00.000000,NaT,NaN,2020-10-05 15:56:11.690041,NaN,NaN,NaT,NaT,nice


User ID: 526


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,3,131,0,526,2019-12-12 20:59:48.763947,NaT,90.0,NaN,money_back,NaN,regular,NaN,3 days 03:00:11.236053,NaT,14 days 02:00:11.236053,NaT,NaT,NaT,NaT,2019-12-16,NaT,2019-12-26 23:00:00,NaT,NaN,2019-12-13 16:32:34.692499,NaN,NaN,NaT,NaT,nice
1,0,614,0,526,2020-02-05 17:54:50.727851,NaT,50.0,NaN,money_back,NaN,regular,NaN,2 days 06:05:09.272149,NaT,22 days 05:05:09.272149,NaT,NaT,NaT,NaT,2020-02-08,NaT,2020-02-27 23:00:00,NaT,NaN,2020-02-06 12:48:34.924114,NaN,NaN,NaT,NaT,nice
2,2,824,0,526,2020-03-08 14:01:05.146330,NaT,60.0,NaN,money_back,NaN,regular,NaN,1 days 09:58:54.853670,NaT,7 days 08:58:54.853670,NaT,-10 days +23:19:15.010974,NaT,NaT,2020-03-10,2020-03-24 23:40:44.989026,2020-03-15 23:00:00,NaT,NaN,2020-03-08 14:57:10.702615,NaN,NaN,NaT,NaT,nice
3,5,985,0,526,2020-03-25 02:27:41.780578,NaT,90.0,NaN,money_back,NaN,regular,NaN,1 days 21:32:18.219422,NaT,19 days 19:32:18.219422,NaT,-9 days +01:09:05.843302,NaT,NaT,2020-03-27,2020-04-22 20:50:54.156698,2020-04-13 22:00:00,NaT,NaN,2020-03-25 11:00:18.003849,NaN,NaN,NaT,NaT,nice
4,4,1348,0,526,2020-04-23 18:54:05.810157,NaT,80.0,NaN,money_back,NaN,regular,NaN,3 days 05:05:54.189843,NaT,17 days 03:05:54.189843,NaT,-12 days +08:11:47.735425,NaT,NaT,2020-04-27,2020-05-22 13:48:12.264575,2020-05-10 22:00:00,NaT,NaN,2020-04-24 08:41:52.485739,NaN,NaN,NaT,NaT,completed
5,6,3478,0,526,2020-06-16 12:38:38.140763,NaT,80.0,NaN,money_back,NaN,regular,NaN,1 days 11:21:21.859237,NaT,12 days 09:21:21.859237,NaT,-2 days +13:13:24.063854,NaT,NaT,2020-06-18,2020-06-30 08:46:35.936146,2020-06-28 22:00:00,NaT,NaN,2020-06-16 13:26:21.815821,NaN,NaN,NaT,NaT,nice
6,7,7250,1904,526,2020-07-08 22:46:36.854214,2020-07-24 17:07:26.992132,90.0,5.0,money_back,cancelled,regular,postpone,1 days 01:13:23.145786,NaT,20 days 23:13:23.145786,NaT,-13 days +00:00:00,NaT,NaT,2020-07-10,2020-08-11 22:00:00.000000,2020-07-29 22:00:00,NaT,before,2020-07-09 08:56:37.509660,Postpone Cash Request 7250,nice,2020-07-29 22:00:00,2020-08-10 22:00:00,nice
7,8,7250,2102,526,2020-07-08 22:46:36.854214,2020-07-27 12:52:44.616105,90.0,5.0,money_back,cancelled,regular,postpone,1 days 01:13:23.145786,NaT,20 days 23:13:23.145786,NaT,-13 days +00:00:00,NaT,NaT,2020-07-10,2020-08-11 22:00:00.000000,2020-07-29 22:00:00,NaT,before,2020-07-09 08:56:37.509660,Postpone Cash Request 7250,nice,2020-07-29 22:00:00,2020-08-09 22:00:00,nice
8,9,12484,4815,526,2020-08-14 12:28:42.222162,2020-08-22 15:08:31.470344,90.0,5.0,money_back,cancelled,instant,postpone,3 days 11:31:17.777838,-4 days +11:31:17.835640,14 days 09:31:17.777838,7 days 09:31:17.835640,-11 days +00:00:00,6 days 23:59:59.942198,2020-08-21 12:28:42.164360,2020-08-18,2020-09-08 22:00:00.000000,2020-08-28 22:00:00,NaT,before,2020-08-14 13:05:51.517542,Postpone Cash Request 12484,nice,2020-08-28 22:00:00,2020-09-10 22:00:00,nice
9,10,12484,5132,526,2020-08-14 12:28:42.222162,2020-08-25 16:19:27.021491,90.0,5.0,money_back,cancelled,instant,postpone,3 days 11:31:17.777838,-4 days +11:31:17.835640,14 days 09:31:17.777838,7 days 09:31:17.835640,-11 days +00:00:00,6 days 23:59:59.942198,2020-08-21 12:28:42.164360,2020-08-18,2020-09-08 22:00:00.000000,2020-08-28 22:00:00,NaT,before,2020-08-14 13:05:51.517542,Postpone Cash Request 12484,nice,2020-08-28 22:00:00,2020-09-10 22:00:00,nice


User ID: 12934


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,0,10094,2500,12934,2020-07-30 10:29:06.883916,2020-08-01 09:43:39.183502,50.0,5.0,money_back,accepted,instant,postpone,1 days 13:30:53.116084,-6 days +13:30:53.116778,16 days 11:30:53.116084,9 days 11:30:53.116778,-1 days +00:00:00,6 days 23:59:59.999306,2020-08-06 10:29:06.883222,2020-08-01,2020-08-16 22:00:00.000000,2020-08-15 22:00:00.000000,2020-08-01 09:43:43.408259,before,2020-07-30 10:45:55.890861,Postpone Cash Request 10094,nice,2020-08-13 10:29:25.986,2020-08-15 22:00:00,nice
1,1,10094,2374,12934,2020-07-30 10:29:06.883916,2020-07-30 10:45:56.972019,50.0,5.0,money_back,accepted,instant,instant_payment,1 days 13:30:53.116084,-6 days +13:30:53.116778,16 days 11:30:53.116084,9 days 11:30:53.116778,-1 days +00:00:00,6 days 23:59:59.999306,2020-08-06 10:29:06.883222,2020-08-01,2020-08-16 22:00:00.000000,2020-08-15 22:00:00.000000,2020-08-18 04:20:03.631389,after,2020-07-30 10:45:55.890861,Instant Payment Cash Request 10094,nice,NaT,NaT,nice
2,2,12969,4261,12934,2020-08-18 17:03:16.380648,2020-08-18 17:04:24.513048,50.0,5.0,money_back,accepted,instant,instant_payment,0 days 06:56:43.619352,-7 days +06:56:43.640985,14 days 00:00:13.055352,7 days 00:00:13.076985,-1 days +19:03:29.436000,6 days 23:59:59.978367,2020-08-25 17:03:16.359015,2020-08-19,2020-09-01 22:00:00.000000,2020-09-01 17:03:29.436000,2020-09-03 06:05:37.049392,after,2020-08-18 17:04:23.114327,Instant Payment Cash Request 12969,nice,NaT,NaT,nice
3,3,14683,6204,12934,2020-09-05 11:15:19.626216,2020-09-05 11:18:13.557782,100.0,5.0,money_back,accepted,instant,instant_payment,2 days 12:44:40.373784,-5 days +12:44:40.393945,14 days 00:00:14.434784,7 days 00:00:14.454945,0 days 10:38:36.069102,6 days 23:59:59.979839,2020-09-12 11:15:19.606055,2020-09-08,2020-09-19 00:36:57.991898,2020-09-19 11:15:34.061000,2020-09-19 00:36:58.048016,after,2020-09-05 11:18:12.405354,Instant Payment Cash Request 14683,nice,NaT,NaT,nice
4,4,17478,8857,12934,2020-09-23 13:44:15.487683,2020-09-23 13:44:54.658790,50.0,5.0,money_back,accepted,instant,instant_payment,0 days 10:15:44.512317,-14 days +10:15:44.534063,14 days 00:00:12.044317,0 days 00:00:12.066063,12 days 03:49:18.007404,13 days 23:59:59.978254,2020-10-07 13:44:15.465937,2020-09-24,2020-09-25 09:55:09.524596,2020-10-07 13:44:27.532000,2020-09-25 09:55:09.562910,after,2020-09-23 13:44:53.535180,Instant Payment Cash Request 17478,nice,NaT,NaT,nice
5,5,17791,11369,12934,2020-09-25 09:58:04.442154,2020-10-06 14:39:28.478601,10.0,5.0,money_back,accepted,instant,postpone,0 days 14:01:55.557846,-7 days +14:01:55.579246,39 days 13:01:55.557846,32 days 13:01:55.579246,28 days 08:17:12.508463,6 days 23:59:59.978600,2020-10-02 09:58:04.420754,2020-09-26,2020-10-06 14:42:47.491537,2020-11-03 23:00:00.000000,2020-10-06 14:39:33.571179,before,2020-09-25 09:58:39.706787,Postpone Cash Request 17791,nice,2020-10-09 09:58:18.793,2020-11-03 23:00:00,nice
6,6,17791,9212,12934,2020-09-25 09:58:04.442154,2020-09-25 09:58:41.146192,10.0,5.0,money_back,accepted,instant,instant_payment,0 days 14:01:55.557846,-7 days +14:01:55.579246,39 days 13:01:55.557846,32 days 13:01:55.579246,28 days 08:17:12.508463,6 days 23:59:59.978600,2020-10-02 09:58:04.420754,2020-09-26,2020-10-06 14:42:47.491537,2020-11-03 23:00:00.000000,2020-10-06 14:42:47.523911,after,2020-09-25 09:58:39.706787,Instant Payment Cash Request 17791,nice,NaT,NaT,nice
7,7,20137,12237,12934,2020-10-09 12:38:14.370122,2020-10-09 12:39:53.922936,10.0,5.0,money_back,accepted,instant,instant_payment,0 days 11:21:45.629878,0 days 11:21:21.266119,10 days 00:00:24.363759,10 days 00:00:00,8 days 19:29:13.165421,0 days 00:00:24.363759,2020-10-09 12:38:38.733881,2020-10-10,2020-10-10 17:09:25